## OMEGA-MAPPIN - Map Comparisons

This notebook runs through the analysis comparing different data types. 

Maps are all put into the same space - the 7500 vertices of the template brain. 
- Oscs, slopes, genes and terms are defined for each vertex.
- Anat is defined over ROIs

Data types availabe: electrophys (oscillations and slopes), genes, terms, and anatomical. 

In [1]:
# Import general code from custom module om, and OO code for handling data
import sys
sys.path.append('/Users/thomasdonoghue/Documents/GitCode/omegamappin/')
from om.gen import *
from om.cl import *

# Set plots to display inline
%matplotlib inline

In [ ]:
# Initialize MapComp object and oscillatory bands list
om_maps = MapComp()
osc_bands = ['Theta', 'Alpha', 'Beta', 'LowGamma']

In [ ]:
# Check files that are available
om_maps.check_files()

In [ ]:
# Load oscillation maps - here using the oscillation scores
om_maps.load_meg_maps(osc_file='Group_Osc_Scores', slope_file='Group_Slopes')

### TERM ANALYSIS

In [ ]:
# Load term maps - here using 10 mm spatial estimation
om_maps.load_term_maps('all_terms_r10_gaussian_estimation.csv')

In [ ]:
# Calculate the correlation between oscillation maps and terms
for osc in osc_bands:
    om_maps.calc_corrs('Terms', osc)

In [ ]:
# Print out highest correlated terms for each oscillation band
for osc in osc_bands:
    om_maps.check_corrs('Terms', osc)

In [ ]:
# Plot an example of distribution of R values and P values
om_maps.plot_corrs('Terms', 'Theta')

In [ ]:
# Save calculated osc correlations to file
for osc in osc_bands:
    om_maps.save_corrs('Terms', osc, save_as_npz=True, save_as_csv=True)

In [ ]:
# Calculate correlations with slopes
om_maps.calc_corrs('Terms', 'Slopes')

In [ ]:
# Check the correlations between term values and slope values
om_maps.check_corrs('Terms', 'Slopes')

In [ ]:
# Plot correlation R- and p-vals between terms & slopes
om_maps.plot_corrs('Terms', 'Slopes')

In [ ]:
# Save out correlations between terms and slopes
om_maps.save_corrs('Terms', 'Slopes')

In [ ]:
# Unload terms - terms & genes are quite large. Unload terms since no longer needed.
om_maps.unload_data('Terms')

### GENE ANALYSIS

In [ ]:
# Load Gene maps - note that genes are split up into 3 files. These are concatenated for analysis. 

# List of gene files
gene_files = ['all_real_genes_donor9861_r10_coords1of3_zscored_gaussian_estimation.csv',
              'all_real_genes_donor9861_r10_coords2of3_zscored_gaussian_estimation.csv',
              'all_real_genes_donor9861_r10_coords3of3_zscored_gaussian_estimation.csv']

# Load gene files
om_maps.load_gene_maps(gene_files)

In [ ]:
# Calculate the correlation between oscillation maps and genes
for osc in osc_bands:
    om_maps.calc_corrs('Genes', osc)

In [ ]:
# Print out highest correlated genes for each oscillation band
for osc in osc_bands:
    om_maps.check_corrs('Genes', osc)

In [ ]:
# Save calculated correlations to file
for osc in osc_bands:
    om_maps.save_corrs('Genes', osc, save_as_npz=True, save_as_csv=True)

In [ ]:
# Calculate the correlation between slopes and genes
om_maps.calc_corrs('Genes', 'Slopes')

In [ ]:
# Print out the highest correlated genes for slopes
om_maps.check_corrs('Genes', 'Slopes')

In [ ]:
# Save calculated correlations to file
om_maps.save_corrs('Genes', 'Slopes', save_as_npz=True, save_as_csv=True)

### ANATOMY ANALYSIS

In [2]:
# Initiliaze Map Comp object in ROI space
om_ROI_maps = MapCompROI()

In [3]:
# Load MEG oscillation & slope data
om_ROI_maps.load_meg_maps(osc_file='Group_Osc_Scores', slope_file='Group_Slopes')

In [4]:
# Load MEG ROIs
elec_roi_path = '/Users/thomasdonoghue/Desktop/scout_Desikan-Killiany_68.mat'
om_ROI_maps.load_elec_rois(elec_roi_path)

In [5]:
# Load anatomy data
om_ROI_maps.load_anat_maps('whole_brain_FreeSurferDKT_count.mat')

In [6]:
# Align ROIs from MEG & Anat
om_ROI_maps.align_rois()

In [7]:
# Convert MEG data into ROIs
om_ROI_maps.conv_meg_rois()

In [ ]:
# Compare anatomical connectivity to MEG data
# TO DO!

In [8]:
len(om_ROI_maps.meg_ROI_maps)

5

In [9]:
om_ROI_maps.meg_ROI_maps['Alpha']

array([ 0.00494757,  0.04852274,  0.07243655,  0.05734931,  0.05790434,
        0.09597966,  0.05842532,  0.01843253,  0.08686286,  0.0254928 ,
        0.03542584,  0.02486223,  0.05625448,  0.03472157,  0.06451088,
        0.02442495,  0.04718557,  0.03073238,  0.0384066 ,  0.10730389,
        0.01419367,  0.07747326,  0.07004946,  0.00620973,  0.0165982 ,
        0.01734733,  0.12220154,  0.04003811,  0.07175601,  0.02431841,
        0.01433393,  0.00551694,  0.04947142,  0.08797631,  0.07476924,
        0.04984321,  0.07183216,  0.05211172,  0.01945669,  0.08295582,
        0.02263276,  0.03562551,  0.02447792,  0.04499468,  0.03407308,
        0.06315797,  0.02146759,  0.05474719,  0.02170692,  0.0531676 ,
        0.09355182,  0.01267269,  0.06431805,  0.06089501,  0.00844357,
        0.0139985 ,  0.01684753,  0.10752554,  0.05457345,  0.02184545,
        0.01509397])

In [ ]:
a = len(np.squeeze(om_ROI_maps.roi_verts[0] - 1))

In [ ]:
om_ROI_maps.nROIs

In [ ]:
a = np.array([1])

In [ ]:
b = np.array([a, 2])

In [ ]:
np.zeros(5)

In [ ]:
np.array([])

In [ ]:
a = np.array([], ndmin=5)
np.shape(a)

In [ ]:
np.zeros([1, 2])

In [ ]:
np.zeros([])

In [ ]:
a = np.array([1])
print a.ndim
print a

In [ ]:
a.put?

In [ ]:
np.put?